In [1]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import AzureOpenAIEmbeddings, OpenAIEmbeddings
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv

load_dotenv()


True

In [15]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{query}"""

math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}"""

In [ ]:
embeddings = AzureOpenAIEmbeddings()
prompt_templates = [physics_template, math_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)

# def prompt_router(input):
#     query_embedding = embeddings.embed_query(input["query"])
#     similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
#     most_similar = prompt_templates[similarity.argmax()]
#     print("Using MATH" if most_similar == math_template else "Using PHYSICS")
#     return PromptTemplate.from_template(most_similar)


# chain = (
#     {"query": RunnablePassthrough()}
#     | RunnableLambda(prompt_router)
#     | AzureChatOpenAI()
#     | StrOutputParser()
# )

In [20]:
llm = AzureChatOpenAI()

In [10]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import AzureChatOpenAI
from typing import List

In [7]:
model = AzureChatOpenAI(temperature=0)

In [13]:
# Define your desired data structure.
class Joke(BaseModel):
    output: str = Field(description="Answer to the user query.")
    follow_up: List[str] = Field(description="List of 4 follow up questions related to query and output.")

In [14]:
# And a query intented to prompt a language model to populate the data structure.
m_query = "What is Myopia ?"

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": m_query})

{'output': 'Myopia, also known as nearsightedness, is a common vision condition in which you can see objects near to you clearly, but objects farther away are blurry. It occurs when the shape of your eye causes light rays to bend (refract) incorrectly, focusing images in front of your retina instead of on your retina.',
 'follow_up': ['What causes myopia?',
  'How is myopia diagnosed?',
  'What are the treatment options for myopia?',
  'Can myopia be prevented or its progression slowed?']}

In [3]:

from langchain_core.outputs import LLMResult
from chain.response_generator import get_response
from langchain_core.callbacks import StdOutCallbackHandler
from langchain.callbacks.streaming_stdout_final_only import FinalStreamingStdOutCallbackHandler
from typing import Any
from flask_socketio import send, emit
import time
from flask import Flask
from flask_socketio import SocketIO

c:\Users\Dell\Desktop\Projects\venv\Lib\site-packages\pydantic\_internal\_fields.py:160: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ('settings_',)`.
  warnings.warn(


In [4]:
class MyCallbackHandler(FinalStreamingStdOutCallbackHandler):
    def __init__(self):
        # Initialize instance variables instead of using global variables
        self.content_checker = True
        self.start_stream = False
        self.check_colon = False
        self.content = ""

    def on_llm_new_token(self, token: str, **kwargs: Any) -> Any:
        """Run on new LLM token. Only available when streaming is enabled."""
        if self.content_checker:
            self.content += token

        if self.content_checker and "Final" in self.content:
            self.check_colon = True
            self.content = ""

        if self.check_colon and ":" in self.content:
            self.content_checker = False
            self.start_stream = True
            self.check_colon = False
            self.content = ""
            return
            
        if self.start_stream:
            return token
            # send_ai_res(token)
            # print(token)
        # send_ai_res(token)

    def on_llm_end(self, response: LLMResult, **kwargs: Any) -> None:
        # Reset instance variables at the end of the response
        self.content_checker = True
        self.start_stream = False
        self.content = ""

In [7]:
import asyncio

async def p():
    print("NOP")

c = asyncio.create_task(p())

NOP


In [12]:
handler_1 = MyCallbackHandler()


app = Flask(__name__)
socketio = SocketIO(app, cors_allowed_origins="*")

def run_test_prompt(input_text):
    executor = get_response()
    executor.invoke({
            "input": input_text,
            "chat_history": [],
        },
        {"callbacks": [handler_1]},
    )

    try:
        for tok in handler_1.on_llm_new_token():
            print(tok, end='', flush=True)  # Print without adding a newline, flush to force immediate printing
            yield tok
    except Exception as e:
        print(f"Error in callback: {e}")

for output in run_test_prompt("What is Myopia?"):
    pass 


Running medical tool
Similarity search initiated...
Final content: association between 
: Myopia, commonly known as nearsightedness, is a condition where the eye is longer than normal or the cornea is too curved, causing light rays to focus in front of the retina instead of on it. This results in distant objects appearing blurry while close objects can be seen clearly. The development of myopia can involve complex interactions between genetic and environmental factors. Recent research suggests that vitamin A supplementation may not have an independent effect on myopia, but intraocular processes associated with vitamin A could indirectly contribute to its development. Moreover, variations in gene expression, such as BMP2 gene variations, have been implicated in ocular development and may contribute to high myopia. Particularly, the gene BMP2-inducible kinase (BMP2K, BIKe) has been found to be upregulated during BMP2-induced osteoblast differentiation and may be connected to susceptibili

In [ ]:
def send_ai_res(msg):
    emit('ai_res', msg, broadcast=False, ignore_queue=False)
    socketio.sleep(0)

@socketio.on('start_stream')
def handle_messagex(data):
    print('received message: ' + data)
    emit("message", "this is a message from the server")
    socketio.sleep(0)
    # run_test_prompt("What is Myopia?")

@socketio.on('send_message')
def handle_message(data):
    print('received message from client: ' + data)
    run_test_prompt(data)

socketio.run(app, port=5001)